<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
Brief explanation on our Booter database analysis methodology: <br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

# Analysis of: vstress
#### File originally retrieved from: http://4lz5rmnkd6f63tmm.onion/db/vStress.sql

## * Libraries that I use to analyse data

In [1]:
import pandas as pd
import numpy as np

from io import StringIO
import re

from IPython.display import display,clear_output

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 0: Reading an input Booter database file<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

In [2]:
dumpfile='vStress.sql'

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 1: Adaptation to our Booter database schema<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

## 1. Reading the first 'N' (100) lines of the input Booter database file

In [3]:
lines_to_read=100

with open(dumpfile) as myfile:
    firstlines=myfile.readlines()[0:lines_to_read] #put here the interval you want
    for x in firstlines:
        print(x.strip())

CREATE TABLE `access` (
`id` int(11) NOT NULL AUTO_INCREMENT,
`username` varchar(15) NOT NULL,
`ip` varchar(15) NOT NULL,
`time` int(11) NOT NULL,
PRIMARY KEY (`id`)
) ENGINE=MyISAM AUTO_INCREMENT=69 DEFAULT CHARSET=latin1;
INSERT INTO `access` (`id`, `username`, `ip`, `time`) VALUES
(1, 'Valderino', '99.250.66.168', 1359691847)
,(2, 'thedon', '65.49.14.71', 1359691851)
,(3, 'thedon', '99.250.66.168', 1359691880)
,(4, 'Valderino', '99.250.66.168', 1359691909)
,(5, 'Valderino', '99.250.66.168', 1359731230)
,(6, 'KyleWTF', '81.135.1.63', 1359733974)
,(7, 'Valderino', '99.250.66.168', 1359739644)
,(8, 'scullzyy', '86.180.27.92', 1359749592)
,(9, 'scullzyy', '86.180.27.92', 1359751705)
,(10, 'Valderino', '99.250.66.168', 1359752087)
,(11, 'sinfuliphone', '107.192.95.162', 1359752343)
,(12, 'scullzyy', '86.180.27.92', 1359753798)
,(13, 'scullzyy', '86.180.27.92', 1359754716)
,(14, 'sinfuliphone', '107.192.95.162', 1359754741)
,(15, 'scullzyy', '86.180.27.92', 1359762072)
,(16, 'Valderino', 

## 2. Listing tables that have content inserted into the dump file

In [4]:
def list_tables_with_insert(dumpfile):
    tables = []
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            if line.lower().startswith('insert'):
                table = re.findall(r'`(.*?)`', line)
                tables.append(table[0])
    return tables

In [5]:
tables=enumerate(list_tables_with_insert(dumpfile))

for i, item in tables:
    print(i+1,item)

1 access
2 gateway
3 logs
4 logs2
5 logs3
6 plans
7 skyper
8 users


## How the list above looks like? 

<h2 align='center'>======================================================================
If NOT well-formed SQL dump file then you must first do the following:
======================================================================</h2>

## 3. Removing lines that are not part of the actual content to be analysed

## 4. Splitting and naming tables and columns

<h2 align='center'>======================================================================
Additional functions
======================================================================</h2>

## * Function to read tables from wel-formed SQL database dump

In [19]:
# For well formed SQL database dumps!
def read_inserted_table(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
            if fast_forward:
                continue
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert line)
#             print(data) 
            try:
                newline = data[0].strip('()') #Step2:remove parenthesis
                newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),', '', newline) #Step4: remove commas from the content of columns
                newline=newline.replace('\'','') #Step5: remove single quotes
                newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                sio.write(newline)
                sio.write("\n")
            except IndexError:
                pass
            if line.endswith(';'):
                break
    sio.seek(0)
    return sio

## * Converter functions for formatting content of columns

In [7]:
def tolowercase(text):
    try:
        return text.lower()
    except AttributeError:
        return text 

In [8]:
def strip_singlequote(text):
    try:
        return text.strip('\'')
    except AttributeError:
        return text

In [9]:
def timestamp2datetime(timestamp):
    try:
        return  pd.to_datetime(timestamp,unit='s')
    except AttributeError:
        return timestamp 

In [10]:
def datetimestring2datetime(text):
    try:
        return  pd.to_datetime(text,format='%d-%m-%Y %H:%M')
    except AttributeError:
        return text 

In [11]:
def datetimeinvertedstring2datetime(text):
    try:
        return  pd.to_datetime(text,format='%Y-%m-%d %H:%M:%S')
    except AttributeError:
        return text 

In [12]:
def datetimeampm2datetime(text):
    try:
        return  pd.to_datetime(text,format='%d-%m-%Y %I:%M:%S %p')
    except AttributeError:
        return text 

In [13]:
def datetimemdyampm2datetime(text):
    try:
        return  pd.to_datetime(text,format='%m-%d-%Y %I:%M:%S %p')
    except AttributeError:
        return text 

In [14]:
def strip_singlequote_and_tolowercase(text):
    try:
        return text.strip('\'').lower()
    except AttributeError:
        return text 

In [15]:
def strip_singlequote_and_dateformat2datetime(text):
    try:
        return  pd.to_datetime(text.strip('\''),format='%d-%m-%Y %H:%M')
    except AttributeError:
        return text 

In [16]:
def strip_parenthesisandsemicolon(text):
    try:
        return text.replace(');',"")
    except AttributeError:
        return text 

In [17]:
def strip_insertintologs(text):
    try:
        return int(text.replace('INSERT INTO `logs` VALUES (',""))
    except AttributeError:
        return text 

<h2 align='center'>======================================================================
Adapting EACH existing table
======================================================================</h2>

## 6.1. Table: 'access'

### How this table looks like without modification?

In [20]:
tablename='access'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,id,username,ip,time
0,1,Valderino,99.250.66.168,1359691847
1,2,thedon,65.49.14.71,1359691851
2,3,thedon,99.250.66.168,1359691880
3,4,Valderino,99.250.66.168,1359691909
4,5,Valderino,99.250.66.168,1359731230
5,6,KyleWTF,81.135.1.63,1359733974
6,7,Valderino,99.250.66.168,1359739644
7,8,scullzyy,86.180.27.92,1359749592
8,9,scullzyy,86.180.27.92,1359751705
9,10,Valderino,99.250.66.168,1359752087


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: logins
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type/converter: 
        o time -> timestamp2datetime() 
    - On the column name:
        o ip -> userip
        o time -> date
    - Split columns:
        o 
    - Add required columns:
        o userid

### Modifying table: 'access'

In [22]:
### Loading the column converting columns with predefined functions
df_logins = pd.read_csv(read_inserted_table(dumpfile, tablename),
                        delimiter=",",
                        error_bad_lines=False,
                       converters = {'time':timestamp2datetime})

###Changing names of columns
df_logins.rename(columns = {   'ip': 'userip',
                               'time': 'date'},
                         inplace=True)

###Creating empty columns (with "" [for future string] or np.nan [for future float])
df_logins['userid'] = np.nan


###Showing some lines after adapt the table
df_logins.head(10)

,id,username,userip,date,userid
0,1,Valderino,99.250.66.168,2013-02-01 04:10:47,NaN
1,2,thedon,65.49.14.71,2013-02-01 04:10:51,NaN
2,3,thedon,99.250.66.168,2013-02-01 04:11:20,NaN
3,4,Valderino,99.250.66.168,2013-02-01 04:11:49,NaN
4,5,Valderino,99.250.66.168,2013-02-01 15:07:10,NaN
5,6,KyleWTF,81.135.1.63,2013-02-01 15:52:54,NaN
6,7,Valderino,99.250.66.168,2013-02-01 17:27:24,NaN
7,8,scullzyy,86.180.27.92,2013-02-01 20:13:12,NaN
8,9,scullzyy,86.180.27.92,2013-02-01 20:48:25,NaN
9,10,Valderino,99.250.66.168,2013-02-01 20:54:47,NaN


## 6.2. Table: 'gateway'

### How this table looks like without modification?

In [23]:
tablename='gateway'

# Only displaying the results
pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,email
0,moneybp@hotmail.ca


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: gateways
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o gateway -> gateways
    - On the column type: 
        o    
    - On the column name:
        o 
    - Split columns:
        o 
    - Add required columns:
        o         

### Modifying table: 'gateways'

In [24]:
df_gateways = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False)

###Showing some lines after adapt the table
df_gateways.head(10)

,email
0,moneybp@hotmail.ca


## 6.3. Table: 'logs'

### How this table looks like without modification?

In [25]:
tablename='logs'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,ID,user,ip,port,time,date
0,6,KyleWTF,141.0.175.164,80,30,1359735779
1,55,daan,82.170.52.117,80,1800,1359814645
2,73,Valderino,141.0.175.164,80,60,1359852347
3,74,Valderino,94.23.153.183,22,60,1359854626
4,82,Valderino,109.3.51.11,80,300,1359862422
5,87,buloy,109.163.238.187,80,500,1359866653
6,88,buloy,198.105.222.185,80,500,1359868465
7,89,buloy,109.163.238.187,80,600,1359869076
8,90,buloy,203.124.105.134,80,500,1359869689
9,99,buloy,203.124.105.134,80,500,1359871541


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: attacks
#### Q3: Are there modifications required? 
    - On the table name: 
        o logs => attacks
    - On the column type: 
        o date -> timestamp2datetime()  
    - On the column name:
        o ID -> id
        o user -> username 
        o ip -> targetip 
        o time -> duration
    - Add required columns:
        o userid 
        o targeturl
        o type
    - Split columns:
        o 

### Modifying table: 'logs'

In [26]:
### Loading the column converting columns with predefined functions
df_attacks = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False,
                       converters = {'date':timestamp2datetime})

###Changing names of columns
df_attacks.rename(columns = {   'ID': 'id',
                               'user': 'username',
                            'ip': 'targetip',
                            'time':'duration'},
                         inplace=True)

###Creating empty columns (with "" [for future string] or np.nan [for future float])
df_attacks['userid'] = np.nan
df_attacks['type'] = ""
df_attacks['targeturl'] = ""

###Showing some lines after adapt the table
df_attacks.head(10)

,id,username,targetip,port,duration,date,userid,type,targeturl
0,6,KyleWTF,141.0.175.164,80,30,2013-02-01 16:22:59,NaN,,
1,55,daan,82.170.52.117,80,1800,2013-02-02 14:17:25,NaN,,
2,73,Valderino,141.0.175.164,80,60,2013-02-03 00:45:47,NaN,,
3,74,Valderino,94.23.153.183,22,60,2013-02-03 01:23:46,NaN,,
4,82,Valderino,109.3.51.11,80,300,2013-02-03 03:33:42,NaN,,
5,87,buloy,109.163.238.187,80,500,2013-02-03 04:44:13,NaN,,
6,88,buloy,198.105.222.185,80,500,2013-02-03 05:14:25,NaN,,
7,89,buloy,109.163.238.187,80,600,2013-02-03 05:24:36,NaN,,
8,90,buloy,203.124.105.134,80,500,2013-02-03 05:34:49,NaN,,
9,99,buloy,203.124.105.134,80,500,2013-02-03 06:05:41,NaN,,


## 6.4. Table: 'logs2'

### How this table looks like without modification?

In [27]:
tablename='logs2'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,ID,user,ip,port,time,date,method
0,2,thedon,202.169.44.143,80,1800,1359692183,UDP
1,3,Valderino,99.250.66.168,80,15,1359692796,UDP
2,4,Valderino,99.250.66.168,80,300,1359692842,UDP
3,5,thedon,202.162.219.74,80,30,1359697163,UDP
4,6,KyleWTF,141.0.175.164,80,30,1359735779,UDP
5,7,scullzyy,81.35.199.161,80,100,1359750071,UDP
6,8,scullzyy,87.79.203.195,80,100,1359751094,UDP
7,10,sinfuliphone,69.162.112.114,25565,60,1359756599,SSYN
8,11,sinfuliphone,69.162.112.114,25565,120,1359756738,SSYN
9,12,sinfuliphone,69.162.112.114,80,120,1359756798,SSYN


#### Q1: Is this table different from other previous loaded table? YES BUT append with attacks
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: attacks
#### Q3: Are there modifications required? 
    - On the table name: 
        o logs2 => attacks_temp
    - On the column type: 
        o date -> timestamp2datetime()  
    - On the column name:
        o ID -> id
        o user -> username 
        o ip -> targetip 
        o time -> duration
        o method -> type
    - Add required columns:
        o userid 
        o targeturl
    - Split columns:
        o 

### Modifying table: 'logs2'

In [28]:
### Loading the column converting columns with predefined functions
df_attacks_temp = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False,
                       converters = {'date':timestamp2datetime})

###Changing names of columns
df_attacks_temp.rename(columns = {   'ID': 'id',
                               'user': 'username',
                            'ip': 'targetip',
                            'time':'duration',
                            'method': 'type'},
                         inplace=True)

###Creating empty columns (with "" [for future string] or np.nan [for future float])
df_attacks_temp['userid'] = np.nan
df_attacks_temp['targeturl'] = ""

###Showing some lines after adapt the table
df_attacks_temp.head(10)

,id,username,targetip,port,duration,date,type,userid,targeturl
0,2,thedon,202.169.44.143,80,1800,2013-02-01 04:16:23,UDP,NaN,
1,3,Valderino,99.250.66.168,80,15,2013-02-01 04:26:36,UDP,NaN,
2,4,Valderino,99.250.66.168,80,300,2013-02-01 04:27:22,UDP,NaN,
3,5,thedon,202.162.219.74,80,30,2013-02-01 05:39:23,UDP,NaN,
4,6,KyleWTF,141.0.175.164,80,30,2013-02-01 16:22:59,UDP,NaN,
5,7,scullzyy,81.35.199.161,80,100,2013-02-01 20:21:11,UDP,NaN,
6,8,scullzyy,87.79.203.195,80,100,2013-02-01 20:38:14,UDP,NaN,
7,10,sinfuliphone,69.162.112.114,25565,60,2013-02-01 22:09:59,SSYN,NaN,
8,11,sinfuliphone,69.162.112.114,25565,120,2013-02-01 22:12:18,SSYN,NaN,
9,12,sinfuliphone,69.162.112.114,80,120,2013-02-01 22:13:18,SSYN,NaN,


In [29]:
df_attacks=df_attacks.append(df_attacks_temp)

## 6.5. Table: 'logs3'

### How this table looks like without modification?

In [30]:
tablename='logs3'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,ID,user,ip,port,time,date,method
0,1,thedon,202.169.44.143,80,1800,1359692183,UDP
1,2,Valderino,99.250.66.168,80,15,1359692796,UDP
2,3,Valderino,99.250.66.168,80,300,1359692842,UDP
3,4,thedon,202.162.219.74,80,30,1359697163,UDP
4,5,KyleWTF,141.0.175.164,80,30,1359735779,UDP
5,6,scullzyy,81.35.199.161,80,100,1359750071,UDP
6,7,scullzyy,87.79.203.195,80,100,1359751094,UDP
7,8,sinfuliphone,67.236.232.125,88,5,1359755031,UDP
8,9,sinfuliphone,69.162.112.114,25565,60,1359756599,SSYN
9,10,sinfuliphone,69.162.112.114,25565,120,1359756738,SSYN


#### Q1: Is this table different from other previous loaded table? YES BUT append with attacks
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: attacks
#### Q3: Are there modifications required? 
    - On the table name: 
        o logs3 => attacks_temp
    - On the column type: 
        o date -> timestamp2datetime()  
    - On the column name:
        o ID -> id
        o user -> username 
        o ip -> targetip 
        o time -> duration
        o method -> type
    - Add required columns:
        o userid 
        o targeturl
    - Split columns:
        o 

### Modifying table: 'logs3'

In [31]:
### Loading the column converting columns with predefined functions
df_attacks_temp = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False,
                       converters = {'date':timestamp2datetime})

###Changing names of columns
df_attacks_temp.rename(columns = {   'ID': 'id',
                               'user': 'username',
                            'ip': 'targetip',
                            'time':'duration',
                            'method': 'type'},
                         inplace=True)

###Creating empty columns (with "" [for future string] or np.nan [for future float])
df_attacks_temp['userid'] = np.nan
df_attacks_temp['targeturl'] = ""

###Showing some lines after adapt the table
df_attacks_temp.head(10)

,id,username,targetip,port,duration,date,type,userid,targeturl
0,1,thedon,202.169.44.143,80,1800,2013-02-01 04:16:23,UDP,NaN,
1,2,Valderino,99.250.66.168,80,15,2013-02-01 04:26:36,UDP,NaN,
2,3,Valderino,99.250.66.168,80,300,2013-02-01 04:27:22,UDP,NaN,
3,4,thedon,202.162.219.74,80,30,2013-02-01 05:39:23,UDP,NaN,
4,5,KyleWTF,141.0.175.164,80,30,2013-02-01 16:22:59,UDP,NaN,
5,6,scullzyy,81.35.199.161,80,100,2013-02-01 20:21:11,UDP,NaN,
6,7,scullzyy,87.79.203.195,80,100,2013-02-01 20:38:14,UDP,NaN,
7,8,sinfuliphone,67.236.232.125,88,5,2013-02-01 21:43:51,UDP,NaN,
8,9,sinfuliphone,69.162.112.114,25565,60,2013-02-01 22:09:59,SSYN,NaN,
9,10,sinfuliphone,69.162.112.114,25565,120,2013-02-01 22:12:18,SSYN,NaN,


In [32]:
df_attacks=df_attacks.append(df_attacks_temp)

## 6.6 Table: 'plans'

### How this table looks like without modification?

In [33]:
tablename='plans'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,id,power,time,length,price
0,1,1,1800,1,8.99
1,2,1,3600,1,14.99
2,3,1,5400,1,19.99
3,4,1,1800,3,14.99
4,5,1,3600,3,27.99
5,6,1,5400,3,37.99
6,7,1,1800,12,28.99
7,8,1,3600,12,49.99
8,9,1,5400,12,69.99
9,10,2,1800,1,12.99


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: plans
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type:
        o
    - On the column name:
        o id -> planid
        o power -> concurrency
        o time -> maxboottime
    - Add required columns:
        o planname 
        o plandescr        
    - Split columns:
        o 

### Modifying table: 'plans'

In [34]:
### Loading the column converting columns with predefined functions
df_plans = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False)

###Changing names of columns
df_attacks_temp.rename(columns = {   'id': 'planid',
                               'power': 'concurrency',
                            'time': 'maxboottime'},
                         inplace=True)

###Creating empty columns (with "" [for future string] or np.nan [for future float])
df_plans['planname'] = ""
df_plans['plandescr'] = ""

###Showing some lines after adapt the table
df_plans.head(10)

,id,power,time,length,price,planname,plandescr
0,1,1,1800,1,8.99,,
1,2,1,3600,1,14.99,,
2,3,1,5400,1,19.99,,
3,4,1,1800,3,14.99,,
4,5,1,3600,3,27.99,,
5,6,1,5400,3,37.99,,
6,7,1,1800,12,28.99,,
7,8,1,3600,12,49.99,,
8,9,1,5400,12,69.99,,
9,10,2,1800,1,12.99,,


## 6.7. Table: 'skyper'

### How this table looks like without modification?

In [36]:
tablename='skyper'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,id,resolved,ip,customer
0,658,nvalderino,99.250.66.168\r\n,admin1
1,659,NaN,NaN,admin1
2,660,nvalderino,NaN,Valderino
3,661,nvalderino,NaN,Valderino
4,662,m47r4x,NaN,Valderino
5,663,ozrikem,NaN,thedon
6,664,Nvalderino,NaN,Valderino
7,665,nvalderino,NaN,Valderino
8,666,ozrikem,NaN,thedon
9,667,m47r4x,NaN,thedon


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o    
    - On the column name:
        o 
    - Add required columns:
        o 
    - Split columns:
        o 

## 6.8. Table: 'users'

### How this table looks like without modification?

In [37]:
tablename='users'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

,ID,username,password,status,rank,mbt,expire,paid
0,1,Valderino,9947eca40cf35c298a450df0b9d6d4dd8655c5f70f2b53...,0,1,1800,1367452491,1
1,4,thedon,bb2dded845d419ca96ffebe81ba5db1f76cc4494d8a781...,0,0,3600,1364767200,1
2,6,KyleWTF,ee26b0dd4af7e749aa1a8ee3c10ae9923f618980772e47...,0,0,1800,1396303200,1
3,7,Scullzyy,7e134ad5b98777718edf1a27004d899a89c7f668f3c985...,0,0,1200,1388530800,1
4,8,sinfuliphone,ea5d455b091834f2efd4ef6e91e24adbdba82c081fe7b6...,0,0,3600,1388530800,1
5,10,Daan,dc077a3a032280f35550a3a9f46d6851a7e43d4c816fc4...,0,0,1800,1388530800,1
6,11,Crumble,238380b48e564d36e9dae79f0d464e064d2250919876a3...,0,0,1800,1360191600,1
7,12,sickshot135,728db48989c9878bdb727058ae0d0968c5902f488dd9e3...,0,0,1800,1388530800,1
8,14,buloy,22a6f953b47ade42251344fead17930c5f6f7974d87cb1...,0,0,1200,1362178800,1
9,15,brentwhittle,728db48989c9878bdb727058ae0d0968c5902f488dd9e3...,0,0,3600,1420066800,1


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: users
    
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o expire => timestamp2datetime 
    - On the column name:
        o ID -> userid
    - Add required columns:
        o useremail
        o planid
    - Split columns:
        o 

### Modifying table: 'users'

In [38]:
### Loading the column converting columns with predefined functions
df_users = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False,
                      converters = {'expire':timestamp2datetime})

###Changing names of columns
df_users.rename(columns = {'ID': 'userid'},
                         inplace=True)

###Creating empty columns (with "" [for future string] or np.nan [for future float])
df_users['useremail'] = ""
df_users['planid'] = ""

###Showing some lines after adapt the table
df_users.head(10)

,userid,username,password,status,rank,mbt,expire,paid,useremail,planid
0,1,Valderino,9947eca40cf35c298a450df0b9d6d4dd8655c5f70f2b53...,0,1,1800,2013-05-01 23:54:51,1,,
1,4,thedon,bb2dded845d419ca96ffebe81ba5db1f76cc4494d8a781...,0,0,3600,2013-03-31 22:00:00,1,,
2,6,KyleWTF,ee26b0dd4af7e749aa1a8ee3c10ae9923f618980772e47...,0,0,1800,2014-03-31 22:00:00,1,,
3,7,Scullzyy,7e134ad5b98777718edf1a27004d899a89c7f668f3c985...,0,0,1200,2013-12-31 23:00:00,1,,
4,8,sinfuliphone,ea5d455b091834f2efd4ef6e91e24adbdba82c081fe7b6...,0,0,3600,2013-12-31 23:00:00,1,,
5,10,Daan,dc077a3a032280f35550a3a9f46d6851a7e43d4c816fc4...,0,0,1800,2013-12-31 23:00:00,1,,
6,11,Crumble,238380b48e564d36e9dae79f0d464e064d2250919876a3...,0,0,1800,2013-02-06 23:00:00,1,,
7,12,sickshot135,728db48989c9878bdb727058ae0d0968c5902f488dd9e3...,0,0,1800,2013-12-31 23:00:00,1,,
8,14,buloy,22a6f953b47ade42251344fead17930c5f6f7974d87cb1...,0,0,1200,2013-03-01 23:00:00,1,,
9,15,brentwhittle,728db48989c9878bdb727058ae0d0968c5902f488dd9e3...,0,0,3600,2014-12-31 23:00:00,1,,


<h2 align='center'>======================================================================
<br>Final step of the manual part
======================================================================</h2>

## 7. Adding missing tables accordingly to our generic Booter database 

In [43]:
df_attacks= pd.DataFrame(columns=['id','userid','username','targetip','targeturl','duration','port','type','date'])
df_blacklist=pd.DataFrame(columns=['id','ip','note'])
df_friendsenemies=pd.DataFrame(columns=['id','ip','note','userid','type'])
# df_gateways=pd.DataFrame(columns=['email'])
# df_logins=pd.DataFrame(columns=['id','userid','username','userip','date'])
df_payments=pd.DataFrame(columns=['id','userid','username','amountpaid','paymentemail','planid','tid','date'])
df_plans=pd.DataFrame(columns=['planid','planname','plandescr','price','maxboottime','concurrency'])
df_servers=pd.DataFrame(columns=['id','ip'])
# df_settings=pd.DataFrame(columns=['url','sitename','siteemail'])
# df_users=pd.DataFrame(columns=['userid','username','useremail','password','expire','plan'])      
# df_webshells=pd.DataFrame(columns=['id','url','status','lastchecked','attacktype'])

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 2: Data Enrichment<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

### * Depending of the size of the data, this part can take HOURS. I tested for both small and big datasets and it worked. Be pacient. This will pay-off!

## * Libraries needed to retrieve information from external databases

In [ ]:
import subprocess
import cfscrape
from lxml import etree
import os.path
import random
import time 

## * Function to enrich IP addresseswith AS information and country 

In [ ]:
# THANKS TO: team-cymru.org
def iptoasn(iplist_teamcymruformat_filelocation,outputfile):
    cat = subprocess.Popen(['cat', iplist_teamcymruformat_filelocation], 
                            stdout=subprocess.PIPE)
    
    netcat = subprocess.Popen(['netcat', 'whois.cymru.com', '43'],
                              stdin=cat.stdout,
                              stdout=outputfile)
    time.sleep(3) #for some reason the poll does not work! This was the way to overcome the waiting time.
    
    return netcat.stdout      

## * Function to check if an IP address was Tor node in a given moment

In [ ]:
# THANKS TO: exonerator.torproject.org
def WasTorNode(ip, date ):
    url="https://exonerator.torproject.org/?ip="+ip+"&timestamp="+date
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url).content    
    html_tree = etree.HTML(scraped_html)
    result=html_tree.xpath("//h3[@class='panel-title']/text()") # I was looking for <h3 class="panel-title">Result is positive</h3>
    tor_node=True if result == ['Result is positive'] else False
    return tor_node 
# 'date' MUST BE formated as: Year-month-day (2016-03-31)

## * Discovering the middle date of the database

In [ ]:
try:
    try:
        middle_date=(min(df_attacks['date'])+((max(df_attacks['date'])-min(df_attacks['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_logins['date'])+((max(df_logins['date'])-min(df_logins['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_payments['date'])+((max(df_payments['date'])-min(df_payments['date']))/2))
        raise
    except:
        pass
except Exception:
    print("There is no date in the entire dataset")

date_tor_check = middle_date.strftime('%Y-%m-%d')
date_iptoasn_lookup= str(middle_date)
print(date_tor_check)

## * Preparing to Perform IP to ASN info

In [ ]:
df_logins['middledate']=date_iptoasn_lookup
df_attacks['middledate']=date_iptoasn_lookup
df_friendsenemies['middledate']=date_iptoasn_lookup
df_blacklist['middledate']=date_iptoasn_lookup

## 2.1.1  Lookup IP to ASN info of table: logins

In [ ]:
if (os.path.exists('enrichments/logins_iptoasn_out')== False):
    logins_iptoasn_in = open('enrichments/logins_iptoasn_in', 'w+')
    logins_iptoasn_in.write('begin\nverbose\n')
    df_logins[['userip','middledate']].drop_duplicates().to_csv(logins_iptoasn_in,header=False,index=False,sep="\t") 
    logins_iptoasn_in.write('end')
    logins_iptoasn_in.close()

    logins_iptoasn_out = open('logins_iptoasn_out', 'w+')
    iptoasn('logins_iptoasn_in',logins_iptoasn_out)
    logins_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_logins_iptoasn = pd.read_csv('enrichments/logins_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

df_logins_extended= pd.merge(df_logins,
                              df_logins_iptoasn,
                              how = 'left',
                              left_on = 'userip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_logins_extended.rename(columns={'asn':'srcasn', 
                                   'ip':'srcip', 
                                   'bgp_prefix':'srcbgp_prefix', 
                                   'country':'srccountry' ,
                                   'registry':'srcregistry',
                                   'info_date':'srcinfo_date',
                                   'info_request':'srcinfo_request'},
                         inplace=True)


df_logins_extended.head(5)

## 2.1.2  Lookup IP to ASN info of table: attacks

In [ ]:
if (os.path.exists('enrichments/attacks_iptoasn_out')== False):
    attacks_iptoasn_in = open('enrichments/attacks_iptoasn_in', 'w+')
    attacks_iptoasn_in.write('begin\nverbose\n')
    df_attacks[['targetip','middledate']].drop_duplicates().to_csv(attacks_iptoasn_in,header=False,index=False,sep="\t") 
    attacks_iptoasn_in.write('end')
    attacks_iptoasn_in.close()

    attacks_iptoasn_out = open('attacks_iptoasn_out', 'w+')
    iptoasn('attacks_iptoasn_in',attacks_iptoasn_out)
    attacks_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_attacks_iptoasn = pd.read_csv('enrichments/attacks_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_attacks_extended= pd.merge(df_attacks,
                              df_attacks_iptoasn,
                              how = 'left',
                              left_on = 'targetip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_attacks_extended.rename(columns={'asn':'targetasn', 
                                   'ip_y':'targetip', 
                                   'bgp_prefix':'targetbgp_prefix', 
                                   'country_y':'targetcountry' ,
                                   'registry':'targetregistry',
                                   'info_date':'targetinfo_date',
                                   'info_request':'targetinfo_request'},
                         inplace=True)
df_attacks_extended.head(5)

## 2.1.3  Lookup IP to ASN info of table: friendsenemies

In [ ]:
if (os.path.exists('enrichments/friendsenemies_iptoasn_out')== False):
    friendsenemies_iptoasn_in = open('enrichments/friendsenemies_iptoasn_in', 'w+')
    friendsenemies_iptoasn_in.write('begin\nverbose\n')
    df_friendsenemies[['ip','middledate']].drop_duplicates().to_csv(friendsenemies_iptoasn_in,header=False,index=False,sep="\t") 
    friendsenemies_iptoasn_in.write('end')
    friendsenemies_iptoasn_in.close()

    friendsenemies_iptoasn_out = open('friendsenemies_iptoasn_out', 'w+')
    iptoasn('friendsenemies_iptoasn_in',friendsenemies_iptoasn_out)
    friendsenemies_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_friendsenemies_iptoasn = pd.read_csv('enrichments/friendsenemies_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_friendsenemies_extended= pd.merge(df_friendsenemies,
                              df_friendsenemies_iptoasn,
                              how = 'left',
                              left_on = 'ip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_friendsenemies_extended.rename(columns={'asn':'friendsenemiesasn', 
                                   'ip':'friendsenemiesip', 
                                   'bgp_prefix':'friendsenemiesbgp_prefix', 
                                   'country':'friendsenemiescountry' ,
                                   'registry':'friendsenemiesregistry',
                                   'info_date':'friendsenemiesinfo_date',
                                   'info_request':'friendsenemiesinfo_request',
                                   'as_name': 'friendsenemiesas_name'},
                         inplace=True)

## 2.1.4  Lookup IP to ASN info of table: blacklist

In [ ]:
if (os.path.exists('enrichments/blacklist_iptoasn_out')== False):
    blacklist_iptoasn_in = open('enrichments/blacklist_iptoasn_in', 'w+')
    blacklist_iptoasn_in.write('begin\nverbose\n')
    df_blacklist[['ip','middledate']].drop_duplicates().to_csv(blacklist_iptoasn_in,header=False,index=False,sep="\t") 
    blacklist_iptoasn_in.write('end')
    blacklist_iptoasn_in.close()

    blacklist_iptoasn_out = open('blacklist_iptoasn_out', 'w+')
    iptoasn('blacklist_iptoasn_in',blacklist_iptoasn_out)
    blacklist_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_blacklist_iptoasn = pd.read_csv('enrichments/blacklist_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_blacklist_extended= pd.merge(df_blacklist,
                              df_blacklist_iptoasn,
                              how = 'left',
                              left_on = 'ip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_blacklist_extended.rename(columns={'asn':'blacklistasn', 
                                   'ip':'blacklistip', 
                                   'bgp_prefix':'blacklistbgp_prefix', 
                                   'country':'blacklistcountry' ,
                                   'registry':'blacklistregistry',
                                   'info_date':'blacklistinfo_date',
                                   'info_request':'blacklistinfo_request',
                                   'as_name': 'blacklistas_name'},
                         inplace=True)
df_blacklist_extended.head(5)

## 2.2.1. Check if IP was a TOR node for table: login

In [ ]:
date_tor_check

In [ ]:
if len(df_logins['userip'].unique())<1200:
    if (os.path.exists('enrichments/logins_torcheck')== False):
        print("Note: it can take a while to finish...")
        logins_torcheck = open('logins_torcheck', 'w+')
        for i in df_logins['userip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=logins_torcheck)
    #         print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            logins_torcheck.flush()

        logins_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.")
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_logins_torcheck = pd.read_csv('enrichments/logins_torcheck',\
                          delimiter="\s+",\
                          names = ['userip', 'tor'])

## 2.2.2. Check if IP was a TOR node for table: attacks

In [ ]:
if len(df_attacks['targetip'].unique())<1200:
    if (os.path.exists('enrichments/attacks_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_attacks['targetip'].unique())*3,"seconds (in the worst case).")

        attacks_torcheck = open('attacks_torcheck', 'w+')

        for i in df_attacks['targetip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=attacks_torcheck)
            print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            attacks_torcheck.flush()

        attacks_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_attacks_torcheck = pd.read_csv('enrichments/attacks_torcheck',\
                          delimiter="\s+",\
                          names = ['targetip', 'tor'])

## 2.2.3. Check if IP was a TOR node for table: friendsenemies

In [ ]:
if len(df_friendsenemies['ip'].unique()) <1200:
    if (os.path.exists('enrichments/friendsenemies_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_friendsenemies['ip'].unique())*3,"seconds (in the worst case).")

        friendsenemies_torcheck = open('enrichments/friendsenemies_torcheck', 'w+')

        for i in df_friendsenemies['ip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=friendsenemies_torcheck)
        #     print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            friendsenemies_torcheck.flush()

        friendsenemies_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_friendsenemies_torcheck = pd.read_csv('enrichments/friendsenemies_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

## 2.2.4. Check if IP was a TOR node for table: blacklist

In [ ]:
if len(df_blacklist['ip'].unique()) < 1200:
    if (os.path.exists('enrichments/blacklist_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_blacklist['ip'].unique())*3,"seconds (in the worst case).")

        blacklist_torcheck = open('enrichments/blacklist_torcheck', 'w+')

        for i in df_blacklist['ip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=blacklist_torcheck)
    #         print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            blacklist_torcheck.flush()

        blacklist_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_blacklist_torcheck = pd.read_csv('enrichments/blacklist_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

## * Relation between Attack dates and Login dates

In [ ]:
def nearestDate(base_date, date_list):
    nearest={}
    for date in date_list:
        if (base_date.timestamp() - date.timestamp())>=0:
            nearest[base_date.timestamp() - date.timestamp()]= date
    return nearest[min(nearest.keys())]

In [ ]:
#This is the TOTAL number records to be checks!!!!
len(df_attacks)

In [ ]:
df_attacks['nearestlogin']=""
df_attacks['nearestlogin']=pd.to_datetime(df_attacks['nearestlogin'])
    
if len(df_attacks)>0 and len(df_logins)>0:
    #When was the last login of the user that performed attacks
    df_attacks['nearestlogin']=""
    df_attacks['nearestlogin']=pd.to_datetime(df_attacks['nearestlogin'])

    for index, row in df_attacks.head(100).iterrows():
        intermediate_df= df_logins[df_logins['username']==row['username']]
        nearestlogindate= nearestDate(row['date'],intermediate_df['date'])
        df_attacks.set_value(index, 'nearestlogin', nearestlogindate)

        #DEBUGGING
        if index % 1000 == 0:
            print(index,": +1000 records analysed")

In [ ]:
if len(df_attacks['nearestlogin'].value_counts()) >1:
    df_attacks_and_logins = pd.merge(df_attacks_extended,
                                     df_logins_extended,
                                     how = 'left',
                                     left_on = ['username','nearestlogin'],
                                     right_on = ['username','date'])
else:
    df_attacks_and_logins=""

<h2 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 3: Automatic Analysis
<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h3> 

## * Libraries that I use to plot figs

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import *
%matplotlib inline

import seaborn as sns
import itertools

plt.style.use('seaborn-muted')
# plt.rcParams['font.family'] = 'serif'
# plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

## A.1. Number of records per table (part of the generic Booter database schema)

In [ ]:
len(df_users),len(df_logins),len(df_attacks),len(df_payments),len(df_settings),len(df_gateways), len(df_friendsenemies),len(df_blacklist),len(df_webshells),len(df_servers),len(df_plans)

## A.2. Number of users, customers, attackers, and their intersection

In [ ]:
if len(set(df_users['userid'].unique())) > 1:
    users_set=set(df_users['userid'].unique())
else:
    users_set=set(df_users['username'].unique())
    
if len(set(df_logins['userid'].unique())) > 1:
    userslogin_set=set(df_logins['userid'].unique())
else:
    userslogin_set=set(df_logins['username'].unique())

if len(set(df_payments['userid'].unique())) > 1:
    customers_set=set(df_payments['userid'].unique())
else:
    customers_set=set(df_payments['username'].unique())

if len(set(df_attacks['userid'].unique())) > 1:
    attackers_set=set(df_attacks['userid'].unique())
else:
    attackers_set=set(df_attacks['username'].unique())

intersec_customers_attacker=pd.Series(list(customers_set.intersection(attackers_set)))
intersec_users_customers=pd.Series(list(users_set.intersection(customers_set)))
intersec_users_attackers=pd.Series(list(users_set.intersection(attackers_set)))
intersec_users_customers_attackers=pd.Series(list(users_set.intersection(customers_set).intersection(attackers_set)))

In [ ]:
len(users_set),len(userslogin_set),len(customers_set),len(attackers_set)

In [ ]:
fig = plt.figure(figsize=(4,3))
fig.suptitle('Users, Customers & Attackers', fontsize=14)

ax = plt.subplot2grid((1,1), (0,0))

venn=venn3(ax=ax,subsets = {'001':len(attackers_set)-len(intersec_customers_attacker)-len(intersec_users_attackers)+len(intersec_users_customers_attackers), 
                            '010':len(customers_set)-len(intersec_users_customers)-len(intersec_customers_attacker)+len(intersec_users_customers_attackers), 
                            '011':len(intersec_customers_attacker)-len(intersec_users_customers_attackers),
                            '100':len(users_set)-len(intersec_users_customers)-len(intersec_users_attackers)+len(intersec_users_customers_attackers),
                            '101':len(intersec_users_attackers)-len(intersec_users_customers_attackers),
                            '110':len(intersec_users_customers)-len(intersec_users_customers_attackers),
                            '111':len(intersec_users_customers_attackers)},\
          set_labels = ('Users', 'Customers','Attackers'),\
          alpha=1)
try:
    venn.get_patch_by_id('100').set_color('#404096')
except:
    print("*Users set is empty!")  
    
try:
    venn.get_patch_by_id('110').set_color('#DEA73A')
except:
    print("*Customers set is empty!")   

try:
    venn.get_patch_by_id('001').set_color('#D92120')
except:
    print("*Attackers set is empty!")

fig.show()
# fig.savefig('../figs/timeseries_attacks.eps', format='eps', dpi=1200) 

## A.3. Distribution of login times per user

In [ ]:
if len(df_logins['userid'].value_counts()) > 0:
    num_distinct_logins_per_user=df_logins['userid'].value_counts()
else:
    num_distinct_logins_per_user=df_logins['username'].value_counts()

freq_distinct_logins_per_user=num_distinct_logins_per_user.value_counts()
cum_dist_user_logins = np.linspace(0.,1.,len(num_distinct_logins_per_user))
cdf_user_logins = pd.Series(cum_dist_user_logins, index=num_distinct_logins_per_user.sort_values())

In [ ]:
if len(num_distinct_logins_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Login Times by Users:', fontsize=14, y=1.05,x=0.35)
    
    #Plot CDF
    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_logins.plot(ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# logins")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    ax1.set_title("")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_logins_per_user.plot(ax=ax2,kind='pie', 
                                       labels=None, 
                                       legend=False,
                                       startangle=270,
#                                        colors=sns.color_palette()
                                       )
    ax2.set_ylabel("")
    
    
    
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_logins_per_user.values)/(freq_distinct_logins_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_logins_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.5, 1.),
               fontsize=10)
    ####
    
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/login_times.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.3. Number of Users that Login via TOR

In [ ]:
len(df_logins_torcheck[df_logins_torcheck['tor']==True]['userip'].value_counts())

## 5.4. Number of Distinct IP addresses by Users

In [ ]:
if len(df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()) >0:
    num_distinct_ips_per_user=df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()
else:
    num_distinct_ips_per_user=df_logins.groupby(['username','userip']).size().reset_index()['username'].value_counts()
    
freq_distinct_ips_per_user=num_distinct_ips_per_user.value_counts()
cum_dist_user_ips = np.linspace(0.,1.,len(num_distinct_ips_per_user))
cdf_user_ips = pd.Series(cum_dist_user_ips, index=num_distinct_ips_per_user.sort_values())

In [ ]:
if len(num_distinct_ips_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Distinct IP address used by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_ips.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_ips_per_user.plot(ax=ax2,kind='pie',
                                    labels=None,legend=False,
                                       startangle=270,
#                                        colors=sns.color_palette()
                                       )
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_ips_per_user.values)/(freq_distinct_ips_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_ips_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.5, 1.),
               fontsize=10)
    ####
    
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/num_ips_by_users.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.5. Number of Payments by Users

In [ ]:
if len(df_payments['userid'].value_counts())>0:
    num_distinct_payments_per_user=df_payments['userid'].value_counts()
else:
    num_distinct_payments_per_user=df_payments['username'].value_counts()

freq_distinct_payments_per_user=num_distinct_payments_per_user.value_counts()
cum_dist_user_payments = np.linspace(0.,1.,len(num_distinct_payments_per_user))
cdf_user_payments = pd.Series(cum_dist_user_payments, index=num_distinct_payments_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Number of Payments by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# Payment")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_per_user.plot(ax=ax2,kind='pie', 
                                         labels=None,legend=False,
                                         startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_payments_per_user.values)/(freq_distinct_payments_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_payments_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/payments_distribution.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.6. Total Amount of Money Earned

In [ ]:
if len(df_payments['amountpaid']) >0:
    total_earned=df_payments['amountpaid'].values.sum()
    'US$ {:,.2f}'.format(float(total_earned))
else:
    print("Unfortunately, there is no data available!")

## 5.7. Amount of Money Paid by Users

In [ ]:
num_distinct_payments_money_per_user=df_payments['amountpaid'].value_counts()
freq_distinct_payments_money_per_user=num_distinct_payments_money_per_user.value_counts()
cum_dist_user_payments_money = np.linspace(0.,1.,len(num_distinct_payments_money_per_user))
cdf_user_payments_money = pd.Series(cum_dist_user_payments_money, index=num_distinct_payments_money_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_money_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Money Payments by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments_money.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("$")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_money_per_user.plot(ax=ax2,kind='pie', 
                                               labels=None,legend=False,
                                               startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_payments_money_per_user.values)/(freq_distinct_payments_money_per_user.values.sum())
    labels = ['${0:1.2f} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_payments_money_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.6, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/distribution_amount_paid.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.8. Countries from where Users Access Booters

In [ ]:
logins_country_distribution_sorted = df_logins_iptoasn['country'].value_counts(sort=True,ascending=True)
logins_country_distribution = df_logins_iptoasn['country'].value_counts()

In [ ]:
if len(logins_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Users Accessing from Countries:', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    logins_country_distribution_sorted.plot(ax=ax1,kind='barh')
    ax1.set_ylabel("# access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    logins_country_distribution.plot(ax=ax2,kind='pie', 
                                     labels=None,legend=False,
                                     startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*logins_country_distribution.values)/(logins_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(logins_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/user_countries.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.9. Countries of Blacklisted IPs

In [ ]:
blacklist_country_distribution=df_blacklist_iptoasn['country'].value_counts()
blacklist_country_distribution_sorted=df_blacklist_iptoasn['country'].value_counts(sort=True,ascending=True)

In [ ]:
if len(blacklist_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Countries of blacklisted IPs', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    blacklist_country_distribution_sorted.plot(ax=ax1,kind='barh')
    ax1.set_ylabel("# Access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    blacklist_country_distribution.plot(ax=ax2,kind='pie', 
                                        labels=None,legend=False,
                                        startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*blacklist_country_distribution.values)/(logins_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(blacklist_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/blacklist_countries.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.10. Countries of Target IPs

In [ ]:
attacks_country_distribution=df_attacks_iptoasn['country'].value_counts()

In [ ]:
if len(attacks_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Countries of target IPs', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    attacks_country_distribution.plot(ax=ax1,kind='bar')
    ax1.set_ylabel("# Access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    attacks_country_distribution.plot(ax=ax2,kind='pie', 
                                      labels=None,legend=False,
                                      startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*attacks_country_distribution.values)/(attacks_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(attacks_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/attack_countries_distribution.eps',bbox_inches='tight', format='eps', dpi=1200) 
else:
    print("Unfortunately, there is no data available!")

## 5.12. Attacks on Same Target

In [ ]:
num_attacks_on_sametarget=df_attacks['targetip'].value_counts()

freq_num_attacks_on_sametarget=num_attacks_on_sametarget.value_counts()
cum_num_attacks_on_sametarget = np.linspace(0.,1.,len(num_attacks_on_sametarget))
cdf_num_attacks_on_sametarget = pd.Series(cum_num_attacks_on_sametarget, index=num_attacks_on_sametarget.sort_values())

In [ ]:
if len(num_attacks_on_sametarget)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Attacks on Same Targets:', fontsize=14, y=1.05,x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_num_attacks_on_sametarget.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_num_attacks_on_sametarget.plot(ax=ax2,kind='pie',
                                        labels=None,legend=False,
                                        startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_num_attacks_on_sametarget.values)/(freq_num_attacks_on_sametarget.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_num_attacks_on_sametarget.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()

    fig.show()
    fig.savefig('figs/attacks_on_same_target.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.11. Attacks per day (timeseries) 

In [ ]:
if len(df_attacks)>0:
    attack_timeseries=df_attacks.set_index(['date']).groupby(pd.TimeGrouper(freq='D')).agg(['count'])['action']
    attack_mean_perday=attack_timeseries.mean()
    attack_median_perday=attack_timeseries.median()
else:
    attack_timeseries=""

In [ ]:
if len(attack_timeseries)>0:
    fig = plt.figure(figsize=(6,3))

    ax1 = plt.subplot2grid((1,1), (0,0), rowspan=2)
    attack_timeseries.plot(ax=ax1,
                           lw=2,
                           legend=False,
    #                        style='--'
                          )

    # X and Y Labels and Ticks
    ax1.set_xlabel("Time (bin=day)")
    ax1.set_ylabel("# Attacks")

    ax1.annotate(str(int(attack_median_perday[0]))+' (median)', 
                 (min(df_attacks['date']), attack_median_perday),
                 xytext=(350, -1), 
                 textcoords='offset points',
                 color='black', 
                 arrowprops=dict(arrowstyle='-|>',
                                 color='black'))
    fig.savefig('figs/attacks_timeseries.eps', bbox_inches='tight',format='eps', dpi=1200)

else:
    print("Unfortunately, there is no data available!")

## 5.12. Time Between Logins and Attacks

In [ ]:
if len(df_attacks_and_logins)>0:
    print("redo")
else:
    print("Unfortunately, there is no data available!")

## 5.13. Who attack whom? (users on the country level)

In [ ]:
if len(df_attacks_extended)>0 and len(df_logins_extended)>0:
    merged_attacks_logins = pd.merge(df_attacks_extended,
                                  df_logins_extended,
                                  how = 'left',
                                  left_on = 'date',
                                  right_on = 'date')[['targetcountry','srccountry']]

    who_against_whom = merged_attacks_logins.groupby(['targetcountry','srccountry'])\
                            .size()\
                            .reset_index()\
                            .pivot('srccountry','targetcountry',0)
else:
    who_against_whom =""

In [ ]:
if len(who_against_whom)>0:
    fig = plt.figure(figsize=(8,12))
    fig.suptitle('Countries of Target IPs', fontsize=14, y=.92)

    ax1 = plt.subplot2grid((1,1), (0,0))
    sns.set()
    sns.heatmap(who_against_whom,
                ax=ax1,
#                 cmap="YlGnBu",
    #             linewidths=.5,
    #             annot=True
                )

    ax1.set_ylabel("Source Country")
    ax1.set_xlabel("Attack Target Country")

    fig.show()
    fig.savefig('figs/who_attack_whom.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

<h1 align="center">!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
FIFTH PART: Query Interface<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

In [ ]:
from ipywidgets import widgets, interact, Dropdown,HTML
from IPython.display import display,clear_output

In [ ]:
def search_by_userid_submit(sender):
    clear_output()
    print("Searching by id =",search_by_userid.value)
    
    if len(df_users[df_users['userid']== int(search_by_userid.value)])>0:
        display(df_users[df_logins['userid']== int(search_by_userid.value)])
    
    if len(df_logins_extended[df_logins_extended['userid']== int(search_by_userid.value)])>0:
        display(df_logins_extended[df_logins_extended['userid']== int(search_by_userid.value)])
    
    if len(df_attacks_extended[df_attacks_extended['userid']== int(search_by_userid.value)])>0:
        display(df_attacks_extended[df_attacks_extended['userid']== int(search_by_userid.value)])
    
    if len(df_payments[df_payments['userid']== int(search_by_userid.value)])>0:
        display(df_payments[df_payments['userid']== int(search_by_userid.value)])

def search_by_username_submit(sender):
    clear_output()
    print("Searching by username =",search_by_username.value,"\n")
    
    if len(df_users[df_users['username']== search_by_username.value])>0:
        print("Table df_users:")
        display(df_users[df_users['username']== search_by_username.value])
    
    if len(df_logins_extended[df_logins_extended['username']== search_by_username.value])>0:
        print("Table df_logins:")
        display(df_logins_extended[df_logins_extended['username']== search_by_username.value])

    if len(df_attacks_extended[df_attacks_extended['username']== search_by_username.value])>0:
        print("Table df_attacks:")
        display(df_attacks_extended[df_attacks_extended['username']== search_by_username.value])
    
    if len(df_payments[df_payments['username']== search_by_username.value])>0:
        print("Table df_payments:")
        display(df_payments[df_payments['username']== search_by_username.value])
        
def search_by_ip_submit(sender):
    clear_output()
    print("Searching by IP address =",str(search_by_ip.value),"\n")
    
    if len(df_logins_extended[df_logins_extended['userip']== str(search_by_ip.value)])>0:
        print("Table df_logins (as ATTACKER:")
        display(df_logins_extended[df_logins_extended['userip']== str(search_by_ip.value)])
     
    if len(df_attacks_extended[df_attacks_extended['targetip']== str(search_by_ip.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetip']== str(search_by_ip.value)])  
      
       
def search_by_asn_submit(sender):
    clear_output()
    print("Searching by Autonomous System Number (ASN) =",search_by_asn.value,"\n")
    
    if len(df_logins_extended[df_logins_extended['srcasn']== int(search_by_asn.value)])>0:
        print("Table df_logins (as ATTACKER):")
        display(df_logins_extended[df_logins_extended['srcasn']== int(search_by_asn.value)])
    
    if len(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])

    if len(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])

        
country_list=pd.read_csv('https://raw.githubusercontent.com/datasets/country-list/master/data.csv',delimiter=",",error_bad_lines=False)
def search_by_country_submit():
    country_code=country_list[country_list['Name']==search_by_country.value]['Code'].values[0]
    clear_output()
    print("Searching by Country =",search_by_country.value,"\n")
    
    if len(df_logins_extended[df_logins_extended['srccountry']== country_code])>0:
        print("Table df_logins (as ATTACKER):")
        display(df_logins_extended[df_logins_extended['srccountry']== country_code])
    
    if len(df_attacks_extended[df_attacks_extended['targetcountry']== country_code])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetcountry']== country_code])
    

In [ ]:
display(HTML('<h3>*Search by (only) one field per time:</h3>'))

search_by_username = widgets.Text(description="username")
display(search_by_username)
search_by_username.on_submit(search_by_username_submit)

search_by_userid = widgets.Text(description="userid")
display(search_by_userid)
search_by_userid.on_submit(search_by_userid_submit)

search_by_ip = widgets.Text(description="IP")
display(search_by_ip)
search_by_ip.on_submit(search_by_ip_submit)

search_by_asn = widgets.Text(description="ASN")
display(search_by_asn)
search_by_asn.on_submit(search_by_asn_submit)

search_by_country = Dropdown(description="Country", options=country_list['Name'].tolist())
search_by_country.on_trait_change(search_by_country_submit, name="value")
display(search_by_country)